# Gabriel:
    Para evaluar el desempeño de los modelos haz lo siguiente:
    Ejecuta las importaciones del Notebook y las funciones cambiar_de_a_poco y counterizar
    Lee tu dataframe
    Aplica la función cambiar_de_a_poco
    Importa los dos modelos
    Evalúa los modelos dándoles como conjunto de X's , la función counterizar.
    

##### IMPORTAR LOS MODELOS DE LA SGTE MANERA:

In [1]:
import pickle

modelo_cargado = pickle.load(open("Bayes_MJpR.sav", "rb"))

modelo_cargado2 = pickle.load(open("Reglog_MJpR.sav", "rb"))

/Users/moisesquiroz/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator BernoulliNB from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/moisesquiroz/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# El objetivo general de esta prueba es alcanzar el mejor desempeño posible para clasificar si un tweet es positivo o negativo.
## Para medir el desempeño, se evaluará con un conjunto de datos del cuál no tendrán acceso. De esta manera evitaremos que los modelos aprendan información sobre el conjunto de validación.
### Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook, archivos auxiliares y csv). Una vez terminada la prueba, comprime la carpeta y sube el .zip a la sección correspondiente.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
import random
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stap = stopwords.words("english")
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/moisesquiroz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/moisesquiroz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import string

In [4]:
import nltk
nltk.download()

#Descargar wordnet desde la pestaña all packages

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# 1.-Generar un análisis exploratorio sobre los datos contenidos en el DataFrame, considerando palabras más comunes y distribución de las clases en el vector objetivo. 

LO QUE HAREMOS, SERÁ CREAR MODELOS DE REGRESIÓN LOGÍSTICA, BAYES, ARBOLES DE DECISIÓN, RANDOMFORESTCLASS, SVM, Adaboost Y Gradiente BUSCANDO LOS MEJORES HIPERPARÁMETROS CON GRIDSEARCH. 

In [5]:
nombre_archivo= input("Ingrese nombre archivo aqui: ")

Ingrese nombre archivo aqui: training_tweets.csv


In [6]:
df = pd.read_csv(nombre_archivo).drop(columns=["Unnamed: 0"])

##### 3. Preparación del vector objetivo y las matrices de entrenamiento y validación:


In [11]:
def cambiar_de_a_poco(Dataframe, col):
    """
    La funcion toma la columna de textos 'col' y mediante regex quita las palabras comunes del ingles, lo transforma
    a minuscula, quita urls, Usuarios de twitter, hashtags, puntuacion y dobles espacios. Finalmente lemantiza el texto
    
    """
    from nltk.corpus import stopwords
    stap = stopwords.words("english")
    Dataframe['sentiment'] = Dataframe['sentiment'].map({'love':1,'worry':-1,'happiness':1,'fun':1,'hate':-1,'sadness':-1,'empty':-1,'relief':1,'surprise':1,'enthusiasm':1,'boredom':-1,'anger':-1})
    Dataframe[col] = Dataframe[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stap)])) #Saca stopwords
    Dataframe[col] = Dataframe[col].apply(lambda x: ' '. join([word.lower() for word in x.split()])) #minúsculas
    Dataframe[col] = pd.Series(map(lambda x: re.sub("((www\.[^\s]+)|(https?://[^\s]+))", "", x), Dataframe[col])) #Convierte url's 
    Dataframe[col] = pd.Series(map(lambda x: re.sub("@[^\s]+", "", x), Dataframe[col])) #Elimina usuarios
    Dataframe[col] = pd.Series(map(lambda x: re.sub(r"#([^\s]+)", r"\1", x), Dataframe[col])) #Saca los Hashtags
    Dataframe[col] = pd.Series(map(lambda x: re.sub('[\s]+', ' ', x), Dataframe[col]))  #Elimina dobles espacios
    Dataframe[col] = pd.Series(map(lambda x: "".join([char for char in x if char not in string.punctuation]), Dataframe[col])) #Eliminar puntuación
    Dataframe[col] = pd.Series(map(lambda x: "".join([char for char in x if char not in string.punctuation]), Dataframe[col])) #Eliminar puntuación
    Dataframe[col] = Dataframe[col].apply(lambda x: ' '. join([lmtzr.lemmatize(word) for word in x.split()])) #Generaliza palabras
    df2 = Dataframe.drop(index=Dataframe[Dataframe.sentiment.isna()].index)
    return df2


In [12]:
df = cambiar_de_a_poco(df, "content")
df.head()

,content,sentiment
0,happy mama day mother,1.0
1,i lost please help find good home,-1.0
2,yes yes i am networking whore fullestand girl ...,1.0
4,microeconomics project ihate subject amp besid...,-1.0
6,first thought bar life meant partying nonstop ...,1.0


In [13]:
def counterizar(DataFrame, colname):
    countvec = CountVectorizer(stop_words=stap, lowercase=True)
    countvec_fit = countvec.fit_transform(DataFrame[colname])
    return countvec_fit

In [14]:
print(classification_report(df["sentiment"],modelo_cargado.predict(counterizar(df,"content")),digits=5))

              precision    recall  f1-score   support

        -1.0    0.81192   0.84171   0.82655     12022
         1.0    0.82834   0.79665   0.81219     11527

    accuracy                        0.81965     23549
   macro avg    0.82013   0.81918   0.81937     23549
weighted avg    0.81996   0.81965   0.81952     23549



In [15]:
print(classification_report(df["sentiment"],modelo_cargado2.predict(counterizar(df,"content")),digits=5))

              precision    recall  f1-score   support

        -1.0    0.78865   0.80236   0.79545     12022
         1.0    0.79007   0.77574   0.78284     11527

    accuracy                        0.78933     23549
   macro avg    0.78936   0.78905   0.78914     23549
weighted avg    0.78935   0.78933   0.78928     23549

